# * Ex-D Data

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [9]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20250201
print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        CASE WHEN REGEXP_LIKE(METRIC_CD, '^DB1R000900|^DB1R001000|^DB1S000101|^DB1S000109|^DB2R000500|^DB2S000100') 
            OR METRIC_CD IN (
                'DB0R00010001' --Total Inflow M1 : DTAC'
                , 'DB1S000100' --Prepay New sub
                , 'DB1S000102' --Prepaid Activation Subs : Thai Mass : DTAC
                , 'DB1S000103' --Prepaid Activation Subs : AEC/Migrants : DTAC
                , 'DB1S000104' --Prepaid Activation Subs : Tourists (IN + OUT) : DTAC
                , 'DB2S010100' --Postpaid activation sub B2C
                , 'DB2S020100' --Postpaid activation sub B2B
                ) THEN 'Obsolete'
            ELSE 'Active'
            END KPI_FLAG
        , PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
        , CASE WHEN REGEXP_LIKE(METRIC_CD, 'A[A-K]$') THEN SUBSTR(METRIC_CD, -2) ELSE 'ALL' END CHANNEL_CD
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) START_DAY, MAX(TM_KEY_DAY) END_DAY
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    GROUP BY PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
    --ORDER BY 1, PAR_KEY, BASENAME, METRIC_CD 
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...


TDMDBPR : Connected

   -> Execute query... 2025-03-10, 18:16:07

   -> Summany DataFrame : 127 rows, 20 columns

TDMDBPR : Disconnected


In [23]:
''' Revenue '''

# curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_d_kpi_flag = 'Active'
rev_metric_list = [
    'DB1R000100' #Prepay Revenue
    , 'DB2R000100' #Postpaid Revenue
    , 'DB2R010100' #Postpaid Revenue B2C
    , 'DB2R020100' #Postpaid Revenue B2B
    ]

src_d_rev_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
# src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_d_rev_overall_df = src_d_rev_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_rev_overall_df = src_d_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_rev_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_rev_overall_df[col] = src_d_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_rev_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250308,DB1R000100,Prepay Revenue,2025-03-08,20250101,20250308,"3,998,485,596","7,996,971,192","7,990,419,136","7,990,419,136","3,998,485,596","82,818,986",5,1098,75241
1,20250308,DB2R000100,Postpaid Revenue,2025-03-08,20250101,20250308,"5,978,100,318","11,956,200,635","11,944,843,796","11,944,843,796","5,978,100,318","211,899,932",5,1098,74808
2,20250308,DB2R010100,Postpaid Revenue B2C,2025-03-08,20250101,20250308,"5,417,083,726","10,834,167,452","10,824,596,285","10,824,596,285","5,417,083,726","196,048,366",5,1098,74764
3,20250308,DB2R020100,Postpaid Revenue B2B,2025-03-08,20250101,20250308,"561,016,591","1,122,033,183","1,120,247,512","1,120,247,512","561,016,591","15,851,566",5,1098,48749


In [24]:
''' Subscription '''

src_d_kpi_flag = 'Active'
sub_metric_list = [
    'DB1S000500' #Prepay Usage Subs
    , 'DB1S000600' #Prepay Revenue Subs
    , 'DB2S000500' #Postpaid Active Subs
    , 'DB2S010500' #Postpaid Active Subs B2C
    , 'DB2S020400' #Postpaid Active Sub B2B
    ]

src_d_sub_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
src_d_sub_overall_df = src_d_sub_overall_df.loc[src_d_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_d_sub_overall_df = src_d_sub_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_sub_overall_df = src_d_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_sub_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_sub_overall_df[col] = src_d_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_sub_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250308,DB1S000500,Prepay Usage Subs,2025-03-08,20250101,20250308,"798,898,584","1,597,797,168","1,596,533,525","1,596,533,525","798,898,584","17,416,337",5,1098,75239
1,20250308,DB1S000600,Prepay Revenue Subs,2025-03-08,20250101,20250308,"425,044,621","850,089,242","849,348,381","849,348,381","425,044,621","9,268,401",5,1098,75239
2,20250308,DB2S000500,Postpaid Active Subs,2025-03-08,20250101,20250308,"391,963,368","783,926,736","783,174,963","783,174,963","391,963,368","13,134,689",5,1098,75108
3,20250308,DB2S010500,Postpaid Active Subs B2C,2025-03-08,20250101,20250308,"345,774,367","691,548,734","690,958,123","690,958,123","345,774,367","11,845,176",5,1098,75108
4,20250308,DB2S020400,Postpaid Active Sub B2B,2025-03-08,20250101,20250308,"46,189,001","92,378,002","92,216,840","92,216,840","46,189,001","1,289,513",5,1098,74674
